In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.io import loadmat


In [9]:
data = pd.read_csv('MC_Modeled_spectra.csv')

In [10]:
X, Y = pd.DataFrame(), pd.DataFrame()
for i, column in enumerate(data.columns):
    if column.isnumeric():
        X[column] = data[column]
    elif column == 'Mel' or column == 'BlSDBN':
        Y[column] = data[column]
X.shape

(218700, 63)

In [11]:
class Node():
    
    def __init__(self, feature_index=None, threshold = None, left = None, right = None, var_red =None, value = None):
        self.feature_index= feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        self.value = value
        
        
        

In [12]:
class DecisionTreeRegressor():
    
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
    
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["var_red"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                print("returning decision node")
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["var_red"])
        print("returning leaf node")
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        '''a function to find the best split of the input feature vector'''
        best_split={}
        max_variance_reduction = -float("inf")
        for j, feature_i in enumerate(range(num_features)):
            print(f'current feature {feature_i}, percent total complete {j/num_features * 100}', end = "\r")
            feature_values = dataset[:,feature_i]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values in the data
            num_thresholds = len(possible_thresholds)
            for i, threshold in enumerate(possible_thresholds):
                dataset_left, dataset_right = self.split(dataset, feature_i, threshold)
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:,-1], dataset_left[:,-1],dataset_right[:,-1]
                    #compute variance reduction
                    curr_var_reduction = self.variance_reduction(y, left_y, right_y)
                    if curr_var_reduction > max_variance_reduction:
                        best_split["feature_index"] = feature_i
                        best_split["threshold"] = threshold
                        best_split["dataset_right"] = dataset_right
                        best_split["dataset_left"] = dataset_left
                        best_split["var_red"] = curr_var_reduction
                        max_variance_reduction = curr_var_reduction
        print("found best split")
        return best_split

                
                
    def split(self, dataset, feature_index, threshold):
        '''function to split the data'''
        data_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        data_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return data_left, data_right
    
    def variance_reduction(self, parent, l_child, r_child):
        '''function to compute variance reduction'''
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        reduction = np.var(parent) - (weight_l * np.var(l_child) + weight_r *np.var(r_child))
        return reduction
    
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        val = np.mean(Y)
        return val
                
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.var_red)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        dataset = np.concatenate((X, Y), axis=1)
        print(dataset.shape)
        self.root = self.build_tree(dataset)
        
    def make_prediction(self, x, tree):
        ''' function to predict new dataset '''
        
        if tree.value!=None: return tree.value
        print(x, tree.feature_index)
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):
        ''' function to predict a single data point '''
        predictions = [self.make_prediction(x, self.root) for i, x in X.iterrows()]
        return predictions

In [13]:

Y_Mel = Y['Mel'].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_Mel, test_size=.2, random_state=41)

## trimmed dataset to reduce computation time
X_train = X_train[:1000]
Y_train = Y_train[:1000]
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1000, 63) (1000, 1) (43740, 63) (43740, 1)


In [14]:
regressor = DecisionTreeRegressor(min_samples_split=3, max_depth=4)
regressor.fit(X_train,Y_train)
regressor.print_tree()

(1000, 64)
found best split62, percent total complete 98.412698412698425
found best split62, percent total complete 98.412698412698425
found best split62, percent total complete 98.412698412698425
found best split62, percent total complete 98.412698412698425
found best split62, percent total complete 98.412698412698425
returning leaf node
returning leaf node
returning decision node
found best split62, percent total complete 98.412698412698425
returning leaf node
returning leaf node
returning decision node
returning decision node
found best split62, percent total complete 98.412698412698425
found best split62, percent total complete 98.412698412698425
returning leaf node
returning leaf node
returning decision node
found best split62, percent total complete 98.412698412698425
returning leaf node
returning leaf node
returning decision node
returning decision node
returning decision node
found best split62, percent total complete 98.412698412698425
found best split62, percent total complet

In [15]:

Y_pred = regressor.predict(X_test[:2000]) 
print(np.sqrt(mean_squared_error(Y_test[:2000], Y_pred)))


380     0.17169
390     0.17209
400     0.17597
410     0.17333
420     0.17547
         ...   
960     0.35381
970     0.33883
980     0.33792
990     0.33767
1000    0.34819
Name: 77108, Length: 63, dtype: float64 11
380     0.17169
390     0.17209
400     0.17597
410     0.17333
420     0.17547
         ...   
960     0.35381
970     0.33883
980     0.33792
990     0.33767
1000    0.34819
Name: 77108, Length: 63, dtype: float64 10
380     0.17169
390     0.17209
400     0.17597
410     0.17333
420     0.17547
         ...   
960     0.35381
970     0.33883
980     0.33792
990     0.33767
1000    0.34819
Name: 77108, Length: 63, dtype: float64 2
380     0.17169
390     0.17209
400     0.17597
410     0.17333
420     0.17547
         ...   
960     0.35381
970     0.33883
980     0.33792
990     0.33767
1000    0.34819
Name: 77108, Length: 63, dtype: float64 9
380     0.17169
390     0.17209
400     0.17597
410     0.17333
420     0.17547
         ...   
960     0.35381
970     0.3388

In [16]:
folder = "Data_Mike";
filename = "R_(1measurement)_normalized_NoGap[202,202,79].mat";
folder = folder + "/";
data_file_to_load = folder + filename;
HSI_Data= loadmat(data_file_to_load);
Hyperspectra_data = HSI_Data['R1']
Hyperspectra_data[np.isinf(Hyperspectra_data)] = 0
Hyperspectra_data_resized = Hyperspectra_data[:,:, 0:43:2]; # Get from 510 to 720 nm with 10nm step size
(x_total, y_total, wavelenght) = Hyperspectra_data_resized.shape;
print("Hyperspectral data: x_total", x_total, "y_total", y_total, "wavelenth",wavelenght);
Hyperspectra_data_reshaped = Hyperspectra_data_resized.reshape(x_total*y_total,-1).astype(np.float32)

print("Hyperspectral reshaped size",Hyperspectra_data_reshaped.shape)


Hyperspectral data: x_total 202 y_total 202 wavelenth 22
Hyperspectral reshaped size (40804, 22)


In [17]:
HSI_predictions = regressor.predict(Hyperspectra_data_reshaped)

mean_squared_error(Y_test, y_pred)

AttributeError: 'numpy.ndarray' object has no attribute 'iterrows'